

# Running the Direct Fidelity Estimation (DFE) algorithm
This example walks through the steps of running the direct fidelity estimation (DFE) algorithm as described in these two papers:

* Direct Fidelity Estimation from Few Pauli Measurements (https://arxiv.org/abs/1104.4695)
* Practical characterization of quantum devices without tomography (https://arxiv.org/abs/1104.3835)

Optimizations for Clifford circuits are based on a tableau-based simulator:
* Improved Simulation of Stabilizer Circuits (https://arxiv.org/pdf/quant-ph/0406196.pdf)

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

In [ ]:
# Import Cirq, DFE, and create a circuit
import cirq
from cirq.contrib.svg import SVGCircuit
import examples.direct_fidelity_estimation as dfe

qubits = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.CNOT(qubits[0], qubits[2]),
    cirq.Z(qubits[0]),
    cirq.H(qubits[2]),
    cirq.CNOT(qubits[2], qubits[1]),
)

SVGCircuit(circuit)

In [ ]:
# We then create a sampler. For this example, we use a simulator
# but the code can accept a hardware sampler.
noise = cirq.ConstantQubitNoiseModel(cirq.depolarize(0.1))
sampler = cirq.DensityMatrixSimulator(noise=noise)

In [ ]:
# We run the DFE:
estimated_fidelity, intermediate_results = dfe.direct_fidelity_estimation(
    circuit,
    qubits,
    sampler,
    n_measured_operators=None,  # None=returns all the Pauli strings
    samples_per_term=0,
)  # 0=use dense matrix simulator

print('Estimated fidelity: %.2f' % (estimated_fidelity))

# What is happening under the hood?
Now, let's look at the `intermediate_results` and correlate what is happening in the code with the papers. The definition of fidelity is:
$$
F = F(\hat{\rho},\hat{\sigma}) = \mathrm{Tr} \left(\hat{\rho} \hat{\sigma}\right)
$$
where $\hat{\rho}$ is the theoretical pure state and $\hat{\sigma}$ is the actual state. The idea of DFE is to write fidelity as:
$$F= \sum _i \frac{\rho _i \sigma _i}{d}$$

where $d=4^{\mathit{number-of-qubits}}$, $\rho _i = \mathrm{Tr} \left( \hat{\rho} P_i \right)$, and $\sigma _i = \mathrm{Tr} \left(\hat{\sigma} P_i \right)$. Each of the $P_i$ is a Pauli operator. We can then finally rewrite the fidelity as:

$$F= \sum _i Pr(i) \frac{\sigma _i}{\rho_i}$$

with $Pr(i) = \frac{\rho_i ^2}{d}$, which is a probability-like set of numbers (between 0.0 and 1.0 and they add up to 1.0).

One important question is how do we choose these Pauli operators $P_i$? It depends on whether the circuit is Clifford or not. In case it is, we know that there are "only" $2^{\mathit{number-of-qubits}}$ operators for which $Pr(i)$ is non-zero. In fact, we know that they are all equiprobable with $Pr(i) = \frac{1}{2^{\mathit{number-of-qubits}}}$. The code does detect the Cliffordness automatically and switches to this mode. In case the circuit is not Clifford, the code just uses all the operators.

Let's inspect that in the case of our example, we do see the Pauli operators with equiprobability (i.e. the $\rho_i$):


In [ ]:
for pauli_trace in intermediate_results.pauli_traces:
    print('Probability %.3f\tPauli: %s' % (pauli_trace.Pr_i, pauli_trace.P_i))

Yay! We do see 8 entries (we have 3 qubits) with all the same 1/8 probability. What if we had a 23 qubit circuit? In this case, that would be quite many of them. That is where the parameter `n_measured_operators` becomes useful. If it is set to `None` we return *all* the Pauli strings (regardless of whether the circuit is Clifford or not). If set to an integer, we randomly sample the Pauli strings.

Then, let's actually look at the measurements, i.e. $\sigma_i$:

In [ ]:
for trial_result in intermediate_results.trial_results:
    print(
        'rho_i=%.3f\tsigma_i=%.3f\tPauli:%s'
        % (trial_result.pauli_trace.rho_i, trial_result.sigma_i, trial_result.pauli_trace.P_i)
    )

How are these measurements chosen? Since we had set `n_measured_operators=None`, all the measurements are used. If we had set the parameter to an integer, we would only have a subset to start from. We would then, as per the algorithm, sample from this set with replacement according to the probability distribution of $Pr(i)$ (for Clifford circuits, the probabilities are all the same, but for non-Clifford circuits, it means we favor more probable Pauli strings).

What about the parameter `samples_per_term`? Remember that the code can handle both a sampler or use a simulator. If we use a sampler, then we can repeat the measurements `samples_per_term` times. In our case, we use a dense matrix simulator and thus we keep that parameter set to `0`.

# How do we bound the variance of the fidelity when the circuit is Clifford?
Recall that the formula for DFE is:
$$F= \sum _i Pr(i) \frac{\sigma _i}{\rho_i}$$

But for Clifford circuits, we have $Pr(i) = \frac{1}{d}$ and $\rho_i = 1$ and thus the formula becomes:
$$F= \frac{1}{d} \sum _i \sigma _i$$

If we estimate by randomly sampling $N$ values for the indicies $i$ for $\sigma_i$ we get:
$$\hat{F} = \frac{1}{N} \sum_{j=1}^N \sigma _{i(j)}$$

Using the Bhatia–Davis inequality ([A Better Bound on the Variance, Rajendra Bhatia and Chandler Davis](https://www.jstor.org/stable/2589180)) and the fact that $0 \le \sigma_i \le 1$, we have the variance of:
$$\mathrm{Var}\left[ \hat{F} \right] \le \frac{(1 - F)F}{N}$$

$$\mathrm{StdDev}\left[ \hat{F} \right] \le \sqrt{\frac{(1 - F)F}{N}}$$

In particular, since $0 \le F \le 1$ we have:
$$\mathrm{StdDev}\left[ \hat{F} \right] \le \sqrt{\frac{(1 - \frac{1}{2})\frac{1}{2}}{N}}$$

$$\mathrm{StdDev}\left[ \hat{F} \right] \le \frac{1}{2 \sqrt{N}}$$